In [202]:
import pandas as pd
import numpy as np
from numpy.linalg.lapack_lite import *
import scipy
import matplotlib.pyplot
import tensorflow as tf
from tensorflow import keras, layers

#from sklearn.model_selection import train_test_split

In [203]:
train_data = pd.read_csv('train.csv')

In [204]:
model = tf.keras.Sequential([
    #keras.layers.Rescaling(1./255, input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    keras.layers.Conv2D(32,5, activation='relu', padding='same'),
    keras.layers.MaxPooling2D((2,2), strides = (3,3)), 
    keras.layers.Conv2D(64,3, strides = (3,3), activation='relu', padding='same'),
    keras.layers.MaxPooling2D((2,2), strides = (3,3)), 
    keras.layers.Conv2D(128,3, strides = (3,3), activation='relu', padding='same'),
    keras.layers.MaxPooling2D((2,2), strides = (3,3)),
    keras.layers.Conv2D(128,3, activation='relu', padding='same'),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Softmax()
])



model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [205]:
files = np.load('arrays.npz')

arrays = [files[key] for key in files]

In [206]:
nps=[]

for x in arrays:
    new = np.resize(x,(256,256))
    new = np.asarray(new)
    new = keras.preprocessing.image.img_to_array(new)
    nps.append(new)

In [207]:
ids = train_data.image_id.unique()

labels = []
for id_ in ids:
    df = train_data[train_data.image_id == id_]
    label = df.iloc[0,2]
    #label = tf.io.decode_raw(label, tf.uint8)
    #label = tf.reshape(label, label.shape)
    #label = tf.one_hot(label, 10)
    labels.append(label)

In [208]:
labels = np.stack(labels)
data = np.stack(nps)

In [249]:
type(labels)

numpy.ndarray

In [210]:
X_train, X_test, y_train, y_test = data[:12000], data[12000:], labels[:12000], labels[12000:]

In [211]:
model.fit(X_train, y_train, epochs=5, batch_size=50, validation_split=0.2)

Train on 9600 samples, validate on 2400 samples
Epoch 1/5
9600/9600 [==============================] - 56s 6ms/sample - loss: 1.6023 - acc: 0.6103 - val_loss: 0.8660 - val_acc: 0.7921
Epoch 2/5
9600/9600 [==============================] - 54s 6ms/sample - loss: 1.3305 - acc: 0.6342 - val_loss: 0.7783 - val_acc: 0.7933
Epoch 3/5
9600/9600 [==============================] - 54s 6ms/sample - loss: 1.2225 - acc: 0.6529 - val_loss: 0.7061 - val_acc: 0.8221
Epoch 4/5
9600/9600 [==============================] - 54s 6ms/sample - loss: 1.2043 - acc: 0.6517 - val_loss: 0.7353 - val_acc: 0.8171
Epoch 5/5
9600/9600 [==============================] - 54s 6ms/sample - loss: 1.1727 - acc: 0.6606 - val_loss: 0.7251 - val_acc: 0.8225


In [212]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_59 (Conv2D)           multiple                  832       
_________________________________________________________________
max_pooling2d_54 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_60 (Conv2D)           multiple                  18496     
_________________________________________________________________
max_pooling2d_55 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_61 (Conv2D)           multiple                  73856     
_________________________________________________________________
max_pooling2d_56 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_62 (Conv2D)           multiple                

In [213]:
model.evaluate(X_test, y_test)

3000/3000 [==============================] - 3s 1ms/sample - loss: 0.5265 - acc: 0.8840


[0.5264852770964304, 0.884]

In [214]:
predictions = model.predict(X_test)

In [248]:
predictions[1]

array([1.8030634e-03, 2.0125337e-04, 2.0125337e-04, 7.2789495e-04,
       2.0125337e-04, 2.0125337e-04, 2.0125337e-04, 2.0125337e-04,
       2.0125337e-04, 2.0125337e-04, 2.0125337e-04, 4.6083005e-04,
       2.0125337e-04, 5.1035301e-04, 9.9448532e-01], dtype=float32)

In [246]:
y_test[0]

14

In [159]:
tf.keras.utils.plot_model(model)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
